# Aumentando o tamanho de imagens

Veremos algumas técnicas que podem ser utilizadas para aumentar o tamanho de imagens. 

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

### Aumento de resolução de imagens por interpolação

In [2]:
x = torch.rand(1, 3, 28, 28)

# Podemos definir o tamanho da saída...
y = F.interpolate(x, size=(56, 56), mode='bilinear')
print(y.shape)

# ...ou o fator de escala
y = F.interpolate(x, scale_factor=2., mode='bilinear')
print(y.shape)

torch.Size([1, 3, 56, 56])
torch.Size([1, 3, 56, 56])


A camada Upsample basicamente chama a função F.interpolate:

In [3]:
upsample = nn.Upsample(size=(56, 56), mode='bilinear')
y = upsample(x)
y.shape

torch.Size([1, 3, 56, 56])

### Aumento de resolução de imagens por convolução transposta

Uma forma comum de aumentar o tamanho de imagens é através da chamada *convolução transposta*. No código abaixo, o resultado possui o dobro do tamanho do sinal de entrada:

In [4]:
x = torch.tensor([6, 3, 7, 9, 8, 3, 1], dtype=torch.float32).reshape(1, 1, -1)
w = torch.tensor([1, 2, 3], dtype=torch.float32).reshape(1, 1, -1)

y = F.conv_transpose1d(x, w, stride=2)
print(y.shape)

torch.Size([1, 1, 15])


#### Interpretação da convolução transposta

Mas o que é a convolução transposta? Para entender ela, primeiro lembramos do fato de que uma convolução pode ser representada como uma multiplicação matricial:

In [5]:
x_half = F.conv1d(x, w, stride=2, padding=1)

# Equivalente à convolução acima. Por causa do stride=2, o kernel "pula" 
# duas posições em cada linha
matrix = torch.tensor([[2, 3, 0, 0, 0, 0, 0],
                       [0, 1, 2, 3, 0, 0, 0],
                       [0, 0, 0, 1, 2, 3, 0],
                       [0, 0, 0, 0, 0, 1, 2]], dtype=torch.float32)

#        4x7 @ 7x1 
res = (matrix@x.reshape(7,1)).T
print(x_half)
print(res)

tensor([[[21., 44., 34.,  5.]]])
tensor([[21., 44., 34.,  5.]])


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


A convolução transposta consiste em fazer exatamente a mesma multiplicação, mas utilizando a transposta da matriz w!

In [6]:
x_rec = F.conv_transpose1d(x_half, w, stride=2, padding=1)

#        7x4   @  4x1
res = (matrix.T@x_half.reshape(4,1)).T
print(x_rec)
print(res)

tensor([[[ 42., 107.,  88., 166.,  68., 107.,  10.]]])
tensor([[ 42., 107.,  88., 166.,  68., 107.,  10.]])


Por isso ela recebe o nome de convolução transposta.

Outra forma de interpretar a convolução transposta é pensar que o sinal de entrada é intercalado com valores 0, e uma convolução normal é realizada:

In [7]:
x_filled = torch.zeros(7)
# Preenche índices ímpares com o valor de x_half
x_filled[::2] = x_half
x_filled = x_filled.reshape(1,1,7)
# Aplica a convolução, precisamos inverter os valores de w para dar certo!
# w: [1,2,3] -> [3,2,1]
F.conv1d(x_filled, w.flip(2), padding=1)

tensor([[[ 42., 107.,  88., 166.,  68., 107.,  10.]]])

É como se a convolução fosse realizada com stride=0.5. Como o sinal de entrada foi intercalado com zeros, o filtro desliza 0.5 pixels nos valores originais, ao invés de 1 pixel como seria na convolução normal. Por isso é comum a convolução transposta ser descrita como "fractionally stridded convolution".

Se tivéssemos utilizado a função `F.conv_transpose1d` com `stride=3`, teríamos um sinal de resultado com o triplo do tamanho. Nesse caso é como se tivéssemos aplicado uma convolução com passo 0.33.

#### Uso da convolução transposta em decodificadores

Uma convolução com stride=2 gera um resultado com metade do tamanho. Já uma convolução transposta com stride=2 gera um resultado com o dobro do tamanho. Portanto, essas operações podem ser utilizadas em tarefas de segmentação para reduzir o tamanho da imagem e depois aumentar o tamanho novamente.

In [8]:
x = torch.rand(1, 1, 7)
x_half = F.conv1d(x, w, stride=2, padding=1)
x_rec = F.conv_transpose1d(x_half, w, stride=2, padding=1)
# O tamanho da saída é igual ao tamanho da entrada:
print(x_rec.shape)

torch.Size([1, 1, 7])


O sinal acima possui tamanho ímpar (7). Quando o tamanho é par, temos uma pequena variação do tamanho:

In [9]:
x = torch.rand(1, 1, 8)
x_half = F.conv1d(x, w, stride=2, padding=1)
x_rec = F.conv_transpose1d(x_half, w, stride=2, padding=1)
print(x_rec.shape)

torch.Size([1, 1, 7])


Isso ocorre porque tamanhos diferentes de entrada podem gerar saídas com mesmo tamanho em uma convolução com stride=2. Nesses casos, a função `conv_transpose1d` aceita um parâmetro opcional `output_padding` que pode ser utilizado para calcular o tamanho da saída de forma correta:

In [10]:
x_rec = F.conv_transpose1d(x_half, w, stride=2, padding=1, output_padding=1)
print(x_rec.shape)

torch.Size([1, 1, 8])


### Interpolação vs convolução transposta

Qual método utilizar para aumentar o tamanho de imagens?

**Interpolação**

Vantagens:
* Não possui parâmetros adicionais para treinar;
* A interpolação de vizinhos mais próximos é extremamente eficiente;
* Podemos escolher exatamente o tamanho da saída;

**Convolução transposta**

Vantagens:
* A rede pode aprender o melhor filtro possível para aumentar o tamanho das imagens da base;

Desvantagens:
* Em algumas tarefas, ela pode adicionar parâmetros desnecessários;
* É preciso adicionar uma lógica complicada para garantir que o tamanho da imagem de saída seja igual ao tamanho da imagem de entrada (parâmetro output_padding);